# Import libraries

In [ ]:
import ee
import geemap
import os

# Setup

In [2]:
ee.Authenticate() # Can be commented out after first authentication

True

In [4]:
ee.Initialize(project="ee-helyne")

In [6]:
# Create local data directory
# data_dir = "/home/jovyan/shared-public/poseidon_heatwaves/data/"  
data_dir = "/Users/helyne/code/climatematch/ouaga-urban-heat-drivers/data/"  
ouaga_dem_dir = os.path.join(data_dir, "dem_ouaga")
os.makedirs(ouaga_dem_dir, exist_ok=True)

print(f"Data will be saved to: {ouaga_dem_dir}")

Data will be saved to: /Users/helyne/code/climatematch/ouaga-urban-heat-drivers/data/dem_ouaga


## General preprocessing remarks/thoughts

- **We should upscale everything to the coarsest dependent variable.**
    - Since our target is LST (30m), we should reproject our 10m Sentinel-2 data to 30m. If we downscale LST to 10m, we're kind of creating "fake" temperature details that don't exist. But if we upscale Sentinel to 30m, we're basically just averaging the vegetation, which is statistically valid.

# Download and prep DEM

In [ ]:
# Define Region of Interest (Ouagadougou) rough estimate (buffer 20km radius)
roi = ee.Geometry.Point([-1.5197, 12.3714]).buffer(20000)

# Function to calculate terrain variables
def add_terrain(image):
    # Calculate slope and hillshade on the individual tile
    slope = ee.Terrain.slope(image).rename('Slope') # in degrees
    aspect = ee.Terrain.aspect(image).rename('Aspect')  # in degrees
    hillshade = ee.Terrain.hillshade(image).rename('Hillshade')
    return image.addBands([slope, aspect, hillshade])

# Access the Copernicus DEM Collection & filter to Ouagadougou ROI
dem_collection = (ee.ImageCollection('COPERNICUS/DEM/GLO30')
    .filterBounds(roi)
    .map(add_terrain)
)

# Mosaic the collection to turn it into a single seamless image and clip to ROI
# We select 'DEM' band specifically for terrain calculations
dem_mosaic = dem_collection.mosaic().clip(roi)

# Extract bands for easier handling
elevation = dem_mosaic.select('DEM')
slope = dem_mosaic.select('Slope')
aspect = dem_mosaic.select('Aspect')
hillshade = dem_mosaic.select('Hillshade')

In [ ]:
# Alignment with satellite data
# We need to reproject the DEM (30m) to match the Sentinel-2 grid (10m).

# Load a sample Sentinel-2 image to get its projection info
s2_image = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(roi) \
    .first()

# Get the projection and scale from the Sentinel-2 band (e.g., Blue band 'B2')
s2_proj = s2_image.select('B2').projection()

# Reproject the DEM variables to match Sentinel-2
# 'Bilinear' is good for continuous data like elevation/slope - makes them look smooth instead of blocky
elevation_aligned = elevation.reproject(crs=s2_proj, scale=10).resample('bilinear')
slope_aligned = slope.reproject(crs=s2_proj, scale=10).resample('bilinear')
# Aspect is circular (0=360), so we use 'bicubic' or 'nearest'
aspect_aligned = aspect.reproject(crs=s2_proj, scale=10).resample('bicubic')

# Now DEM data has the same pixel grid as the Sentinel-2 data.

In [ ]:
# Inspect some data
slope_aligned

In [ ]:
# Double check the projection
proj_info = elevation_aligned.projection().getInfo()

print("Coordinate Reference System (CRS):", proj_info['crs'])
print("Pixel Scale (Resolution in meters):", proj_info['transform'][0])

# RESULT INTERPRETATION:
# If CRS is 'EPSG:4326', it is in Lat/Long.
# If CRS is 'EPSG:32630', it is in UTM Zone 30N (common for Burkina Faso).
# If Transform[0] is 0.00027..., it's in degrees (approx 30m).
# If Transform[0] is 10, it's in meters.

Coordinate Reference System (CRS): EPSG:32630
Pixel Scale (Resolution in meters): 10


In [25]:
# Calculate Min, Max, and Mean elevation of region
stats = elevation_aligned.reduceRegion(
    reducer=ee.Reducer.minMax().combine(
        reducer2=ee.Reducer.mean(),
        sharedInputs=True
    ),
    geometry=roi,
    crs=s2_proj, 
    scale=30,  # Native resolution of Copernicus
    bestEffort=True
).getInfo()

print("Elevation Statistics for ROI:")
print(f"Min Elevation: {stats['DEM_min']} meters")
print(f"Max Elevation: {stats['DEM_max']} meters")
print(f"Mean Elevation: {stats['DEM_mean']} meters")

Elevation Statistics for ROI:
Min Elevation: 268.09613037109375 meters
Max Elevation: 362.32373046875 meters
Mean Elevation: 303.173029140881 meters


In [ ]:
# Calculate Min, Max, and Mean slope of region
stats = slope_aligned.reduceRegion(
    reducer=ee.Reducer.minMax().combine(
        reducer2=ee.Reducer.mean(),
        sharedInputs=True
    ),
    geometry=roi,
    crs=s2_proj, 
    scale=30,  # Native resolution of Copernicus
    bestEffort=True
).getInfo()

print("Slope Statistics for ROI:")
print(f"Min Slope: {stats['Slope_min']}")
print(f"Max Slope: {stats['Slope_max']}")
print(f"Mean Slope: {stats['Slope_mean']}")

Slope Statistics for ROI:
Min Slope: 0 meters
Max Slope: 18.325115203857422 meters
Mean Slope: 1.0987662468123665 meters


In [27]:
# Create an interactive map centered on Ouagadougou
Map = geemap.Map(center=[12.3714, -1.5197], zoom=11)

# Set visualization parameters
# Ouagadougou is relatively flat (approx 280m - 380m)
# Setting min/max or the image will just look solid gray
dem_vis = {
    'min': 280,
    'max': 380,
    'palette': ['green', 'yellow', 'brown']
}

slope_vis = {
    'min': 0,
    'max': 10, # It's a flat city, slopes > 10 degrees are rare
    'palette': ['black', 'white']
}

# Add layers to the map
# 'elevation' and 'slope' are variables from the previous step
Map.addLayer(elevation_aligned, dem_vis, 'Elevation (DEM)')
Map.addLayer(slope_aligned, slope_vis, 'Slope')

# Display the map
Map

Map(center=[12.3714, -1.5197], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…